In [1]:
import torch
torch.cuda.device_count()

1

In [1]:
import torch, math, numpy as np
import torch.nn as nn
import torch.optim as optim
import pylab as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        nopeak_mask = nopeak_mask.to(device)   #### ????? #####
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [61]:
src_vocab_size = 4096
tgt_vocab_size = 4096
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 1536
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

src_data = torch.from_numpy(np.load('/share/data/2pals/jim/data/X_450.npy').astype(int))
tgt_data = torch.from_numpy(np.load('/share/data/2pals/jim/data/Y_450.npy').astype(int))

transformer.to(device)
src_data = src_data.to(device)
tgt_data = tgt_data.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

batch_size = 32
batches = 7

for epoch in range(200):
    for batch in range(batches):
        start = int(batch * batch_size)
        end = int(batch * batch_size + batch_size)
        optimizer.zero_grad()
        output = transformer(src_data[start:end,:], tgt_data[start:end, :-1])
        loss   = criterion( output.contiguous().view(-1, tgt_vocab_size), tgt_data[start:end, 1:].contiguous().view(-1) )
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

transformer.eval()
pred = transformer(src_data[:20,:], tgt_data[:20, :-1])
np.save('/share/data/2pals/jim/data/test_predicitons_6_8.npy', np.array(pred.argmax(axis=2).cpu()))

Epoch: 1, Loss: 6.395374774932861
Epoch: 2, Loss: 5.986266136169434
Epoch: 3, Loss: 5.705990314483643
Epoch: 4, Loss: 5.4856343269348145
Epoch: 5, Loss: 5.310023307800293
Epoch: 6, Loss: 5.181400775909424
Epoch: 7, Loss: 5.085275650024414
Epoch: 8, Loss: 5.009516716003418
Epoch: 9, Loss: 4.948401927947998
Epoch: 10, Loss: 4.895383358001709
Epoch: 11, Loss: 4.842586994171143
Epoch: 12, Loss: 4.807000637054443


KeyboardInterrupt: 

In [3]:
src_data = torch.from_numpy(np.load('/share/data/2pals/jim/data/X_450.npy').astype(int))

FileNotFoundError: [Errno 2] No such file or directory: '/share/data/2pals/jim/data/X_450.npy'

In [37]:
tgt_data[:20, 1:].contiguous().view(-1)

torch.Size([8980])

In [54]:
pred.contiguous().view(-1, tgt_vocab_size).shape

torch.Size([8980, 4096])

In [55]:
embeddings = nn.Embedding(8980, 4096)
my_sample = torch.randn(1, 4096)
distance = torch.norm(embeddings.weight.data - my_sample, dim=1)
nearest = torch.argmin(distance)
nearest

tensor(8949)

In [39]:
my_sample.shape

torch.Size([1, 100])

In [60]:
nn.Embedding(8980, 4096).weight.data 

tensor([[ 1.7073e+00,  1.0622e+00, -3.3787e-01,  ...,  5.9104e-01,
         -1.2925e-01,  4.8686e-01],
        [ 1.4781e-01,  7.4789e-01, -6.4973e-01,  ...,  4.9797e-01,
         -7.8069e-01,  1.4190e+00],
        [ 9.6601e-01, -2.4418e-01,  6.2233e-01,  ..., -1.5385e-01,
         -1.7291e+00, -1.2717e+00],
        ...,
        [ 1.0278e+00, -8.3493e-01,  1.0904e+00,  ...,  9.6026e-01,
          1.1136e+00, -2.6322e-02],
        [ 2.8953e-01, -2.1885e-01, -3.8298e-01,  ..., -4.4413e-01,
          3.7046e-01,  8.1144e-01],
        [-4.0478e-01, -1.2435e-01, -1.3952e-03,  ..., -5.3351e-01,
          1.7312e-01,  7.7977e-02]])

In [24]:
embeddings = nn.Embedding(512, 4096)
distance = torch.norm(embeddings.weight.data - pred[1,0,:].cpu(), dim=1)
nearest = torch.argmin(distance)
nearest

tensor(266)

In [13]:
### OLD

In [16]:
# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (20, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (20, max_seq_length))  # (batch_size, seq_length)
src_data.shape

torch.Size([20, 450])

In [3]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(3):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss   = criterion( output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1) )
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.494218826293945
Epoch: 2, Loss: 8.343814849853516
Epoch: 3, Loss: 8.268326759338379
